In [1]:
!pip install zarr xarray xarray[io]
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install google-cloud-storage
#!pip install dask

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import gcsfs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

/opt/conda/lib/python3.9/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [ ]:
'''
from dask.distributed import Client
from dask_gateway import GatewayCluster

cluster = GatewayCluster()
client = Client(cluster)
cluster.scale(2)
'''

In [5]:
from google.cloud import storage

def list_bucket_contents(bucket_name, folder_path=None):
    """Lists the contents of a folder within a Google Cloud Storage bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=folder_path)

    for blob in blobs:
        print(blob.name)

list_bucket_contents("ohw23-oil")

CSKS2.zarr/.zattrs
CSKS2.zarr/.zgroup
CSKS2.zarr/.zmetadata
CSKS2.zarr/band/.zarray
CSKS2.zarr/band/.zattrs
CSKS2.zarr/band/0
CSKS2.zarr/band_data/.zarray
CSKS2.zarr/band_data/.zattrs
CSKS2.zarr/band_data/0.0.0
CSKS2.zarr/band_data/0.0.1
CSKS2.zarr/band_data/0.0.2
CSKS2.zarr/band_data/0.0.3
CSKS2.zarr/band_data/0.0.4
CSKS2.zarr/band_data/0.0.5
CSKS2.zarr/band_data/0.0.6
CSKS2.zarr/band_data/0.0.7
CSKS2.zarr/band_data/0.1.0
CSKS2.zarr/band_data/0.1.1
CSKS2.zarr/band_data/0.1.2
CSKS2.zarr/band_data/0.1.3
CSKS2.zarr/band_data/0.1.4
CSKS2.zarr/band_data/0.1.5
CSKS2.zarr/band_data/0.1.6
CSKS2.zarr/band_data/0.1.7
CSKS2.zarr/band_data/0.10.0
CSKS2.zarr/band_data/0.10.1
CSKS2.zarr/band_data/0.10.2
CSKS2.zarr/band_data/0.10.3
CSKS2.zarr/band_data/0.10.4
CSKS2.zarr/band_data/0.10.5
CSKS2.zarr/band_data/0.10.6
CSKS2.zarr/band_data/0.10.7
CSKS2.zarr/band_data/0.11.0
CSKS2.zarr/band_data/0.11.1
CSKS2.zarr/band_data/0.11.2
CSKS2.zarr/band_data/0.11.3
CSKS2.zarr/band_data/0.11.4
CSKS2.zarr/band_data

In [6]:
fs = gcsfs.GCSFileSystem()
mapper = fs.get_mapper("gs://ohw23-oil/CSKS2.zarr")
data_array = xr.open_zarr(mapper,consolidated=True)

/opt/conda/lib/python3.9/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [7]:
data_array

<xarray.Dataset>
Dimensions:      (band: 1, y: 8316, x: 11199)
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 4.023e+05 4.023e+05 ... 5.702e+05 5.702e+05
  * y            (y) float64 7.834e+06 7.834e+06 ... 7.709e+06 7.709e+06
Data variables:
    band_data    (band, y, x) float32 dask.array<chunksize=(1, 520, 1400), meta=np.ndarray>
    spatial_ref  int32 ...

In [ ]:
#Example of setting a threshold on the data before doing histogram binning
threshold = 2000
ds = data_array.where(data_array < threshold)

In [14]:
#Setup the histogram bins.  In this case, a bin size of size 10
bins = np.linspace(-1, threshold, 10)
print(bins)

#Function to process each chunk.  Example of histogram (not sure if this is the best/fastest way to do it)
def process_chunk(chunk):
  chunk = chunk.to_array()
  print(np.unique(chunk.to_numpy()))
  result = chunk.groupby_bins(chunk, bins).apply(xr.DataArray.count).fillna(0)
  return result

[-1.00000000e+00  2.21333333e+02  4.43666667e+02  6.66000000e+02
  8.88333333e+02  1.11066667e+03  1.33300000e+03  1.55533333e+03
  1.77766667e+03  2.00000000e+03]


In [15]:
#This will create N levels of chunk size (like a pyramid).  
def pyramid(data_array, levels=[500, 250, 100]):
  for chunksize in levels:
    data = data_array.chunk({'y':chunksize,'x':chunksize, 'band':1}).chunks
    ix = data['x']
    iy = data['y']

    idx = [sum(ix[:i]) for i in range(len(ix)+1)]
    idy = [sum(iy[:i]) for i in range(len(iy)+1)]

    for i in range(0, len(idx)-1, 1):
      tgt_x = xr.DataArray(np.arange(idx[i], idx[i+1]), dims="x")
      for j in range(0, len(idy)-1, 1):
        #print(f"{idx[i]}, {idy[j]}")
        tgt_y = xr.DataArray(np.arange(idy[j], idy[j+1]), dims="y")
        chunk = data_array.isel(y=tgt_y, x=tgt_x, band=0)
        do_something = process_chunk(chunk)
        ####
        # Do something with the result of the process_chunk function
        # 
        ####
        

In [17]:
#This will be slow :(
pyramid(ds)

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[   0.  130.  251.  265.  329.  339.  355.  382.  384.  418.  427.  433.
  434.  438.  443.  444.  454.  462.  464.  466.  469.  474.  482.  483.
  486.  488.  515.  518.  525.  528.  532.  533.  534.  535.  542.  555.
  556.  558.  559.  563.  565.  566.  574.  575.  582.  586.  588.  590.
  592.  593.  595.  598.  599.  601.  607.  613.  615.  617.  620.  622.
  624.  625.  628.  636.  637.  638.  639.  640.  641.  649.  653.  656.
  660.  666.  669.  672.  673.  675.  676.  679.  682.  683.  684.  686.
  690.  692.  694.  699.  704.  708.  710.  714.  716.  719.  722.  723.
  724.  725.  726.  728.  729.  730.  732.  733.  734.  735.  740.  741.
  744.  745.  746.  747.  749.  750.  755.  756.  758.  760.  761.  764.
  765.  766.  767.  768.  771.  772.  773.  774.  775.  780.  782.  784.
  792.  793.  796.  797.  798.  801.  802.  804.  805.  807.  808.  810.
  812.  814.  816.  817.  819.  820.  821.  824.  826.  827.  828.  830.
  831.  832.  83

KeyboardInterrupt: 